<a href="https://colab.research.google.com/github/hmg8906/CienciaDatosCN/blob/main/Cob_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Clasificación de coberturas a partir de imágenes satelitales usando Google Earth Engine (GEE) y capa de coberturas 2020 IDEAM
import ee
import geemap

# Inicializar Google Earth Engine
ee.Initialize()

# Cargar la zona de estudio desde el archivo shapefile
roi = geemap.shp_to_ee('ruta/a/tu/archivo/roi.shp')

# Cargar la imagen satelital del año 2020 (por ejemplo, Landsat 8)
imagen_2020 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2020-01-01', '2020-12-31') \
    .median() \
    .clip(roi)

# Cargar la capa de coberturas de la tierra del año 2020 (CORINE o similar)
cobertura_2020 = ee.Image('COPERNICUS/CORINE/V20/100m/2020') \
    .clip(roi)

# Generar puntos aleatorios usando la capa de coberturas 2020 como referencia
puntos_aleatorios = cobertura_2020.stratifiedSample(
    numPoints=5000,  # Número de puntos a generar
    classBand=cobertura_2020.select('landcover'),  # Ajusta según las propiedades de la cobertura
    region=roi,
    scale=30,  # Resolución espacial
    seed=42,
    geometries=True  # Incluir geometrías para uso posterior
)

# Extraer las bandas de la imagen 2020 para usarlas como variables predictoras
bandas = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
entrenamiento = imagen_2020.select(bandas).sampleRegions(
    collection=puntos_aleatorios,
    properties=['landcover'],  # Ajusta según las propiedades de la cobertura
    scale=30
)

# Entrenar un clasificador supervisado (por ejemplo, Random Forest)
clasificador = ee.Classifier.smileRandomForest(100).train(
    features=entrenamiento,
    classProperty='landcover',
    inputProperties=bandas
)

# Cargar la imagen satelital del año 2024
imagen_2024 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2024-01-01', '2024-12-31') \
    .median() \
    .clip(roi)

# Aplicar el modelo de clasificación entrenado a la imagen de 2024
clasificacion_2024 = imagen_2024.select(bandas).classify(clasificador)

# Visualizar el resultado
Map = geemap.Map()
Map.addLayer(clasificacion_2024.randomVisualizer(), {}, 'Clasificación 2024')
Map.addLayer(roi, {}, 'ROI')
Map.centerObject(roi, 10)
Map.show()

# Exportar la clasificación
exportar = ee.batch.Export.image.toDrive(
    image=clasificacion_2024,
    description='clasificacion_supervisada_2024',
    scale=30,
    region=roi.geometry().bounds().getInfo(),
    maxPixels=1e13
)
exportar.start()